# Prediabetes Risk Prediction Analysis

This notebook demonstrates a comprehensive approach to predicting prediabetes risk using machine learning techniques. The analysis includes feature engineering, model development, and evaluation with a focus on healthcare applications.

**Disclaimer**: This analysis uses synthetic data for educational purposes. In real healthcare applications, ensure compliance with HIPAA and other relevant regulations.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Data Generation and Loading

In [ ]:
# Generate synthetic prediabetes data
np.random.seed(42)
n_samples = 10000

# Create synthetic features
age = np.random.normal(55, 15, n_samples)
age = np.clip(age, 18, 85)

bmi = np.random.normal(28, 6, n_samples)
bmi = np.clip(bmi, 16, 50)

fasting_glucose = np.random.normal(110, 25, n_samples)
fasting_glucose = np.clip(fasting_glucose, 70, 200)

hba1c = np.random.normal(5.8, 0.8, n_samples)
hba1c = np.clip(hba1c, 4.0, 8.0)

systolic_bp = np.random.normal(135, 20, n_samples)
systolic_bp = np.clip(systolic_bp, 90, 200)

diastolic_bp = np.random.normal(85, 12, n_samples)
diastolic_bp = np.clip(diastolic_bp, 60, 120)

triglycerides = np.random.lognormal(4.5, 0.5, n_samples)
triglycerides = np.clip(triglycerides, 50, 500)

hdl_cholesterol = np.random.normal(45, 12, n_samples)
hdl_cholesterol = np.clip(hdl_cholesterol, 25, 80)

waist_circumference = np.random.normal(95, 15, n_samples)
waist_circumference = np.clip(waist_circumference, 60, 140)

# Create target variable with realistic prediabetes risk
risk_score = (
    0.3 * (age - 50) / 15 +
    0.25 * (bmi - 25) / 5 +
    0.2 * (fasting_glucose - 100) / 20 +
    0.15 * (hba1c - 5.7) / 0.3 +
    0.1 * (systolic_bp - 120) / 20
)

prediabetes_risk = (risk_score > 0.5).astype(int)

# Create categorical variables
gender = np.random.choice(['Male', 'Female'], n_samples, p=[0.48, 0.52])
family_history = np.random.choice(['Yes', 'No'], n_samples, p=[0.3, 0.7])
physical_activity = np.random.choice(['Low', 'Moderate', 'High'], n_samples, p=[0.4, 0.4, 0.2])
smoking_status = np.random.choice(['Never', 'Former', 'Current'], n_samples, p=[0.6, 0.25, 0.15])

# Create DataFrame
data = pd.DataFrame({
    'age': age,
    'gender': gender,
    'bmi': bmi,
    'fasting_glucose': fasting_glucose,
    'hba1c': hba1c,
    'systolic_bp': systolic_bp,
    'diastolic_bp': diastolic_bp,
    'triglycerides': triglycerides,
    'hdl_cholesterol': hdl_cholesterol,
    'waist_circumference': waist_circumference,
    'family_history': family_history,
    'physical_activity': physical_activity,
    'smoking_status': smoking_status,
    'prediabetes_risk': prediabetes_risk
})

print(f"Dataset shape: {data.shape}")
print(f"Prediabetes risk prevalence: {data['prediabetes_risk'].mean():.2%}")
data.head()

## 3. Exploratory Data Analysis

In [ ]:
# Basic statistics
print("Dataset Overview:")
print("=" * 50)
print(f"Total samples: {len(data)}")
print(f"Features: {data.shape[1] - 1}")
print(f"Prediabetes cases: {data['prediabetes_risk'].sum()}")
print(f"Prevalence: {data['prediabetes_risk'].mean():.2%}")

print("\nData Types:")
print(data.dtypes)

print("\nMissing Values:")
print(data.isnull().sum())

In [ ]:
# Distribution of numerical features
numerical_features = ['age', 'bmi', 'fasting_glucose', 'hba1c', 'systolic_bp', 
                     'diastolic_bp', 'triglycerides', 'hdl_cholesterol', 'waist_circumference']

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(numerical_features):
    axes[i].hist(data[feature], bins=30, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'Distribution of {feature.replace("_", " ").title()}')
    axes[i].set_xlabel(feature.replace("_", " ").title())
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
correlation_matrix = data[numerical_features + ['prediabetes_risk']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Categorical feature analysis
categorical_features = ['gender', 'family_history', 'physical_activity', 'smoking_status']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, feature in enumerate(categorical_features):
    risk_by_category = data.groupby(feature)['prediabetes_risk'].mean()
    axes[i].bar(risk_by_category.index, risk_by_category.values, alpha=0.7)
    axes[i].set_title(f'Prediabetes Risk by {feature.replace("_", " ").title()}')
    axes[i].set_ylabel('Prediabetes Risk Rate')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
# Create additional features
data['age_group'] = pd.cut(data['age'], bins=[0, 35, 50, 65, 100], 
                           labels=['Young', 'Middle', 'Senior', 'Elderly'])
data['bmi_category'] = pd.cut(data['bmi'], bins=[0, 18.5, 25, 30, 100], 
                              labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
data['glucose_category'] = pd.cut(data['fasting_glucose'], bins=[0, 100, 126, 200], 
                                  labels=['Normal', 'Prediabetes', 'Diabetes'])

# Create interaction features
data['age_bmi_interaction'] = data['age'] * data['bmi'] / 1000
data['glucose_hba1c_interaction'] = data['fasting_glucose'] * data['hba1c'] / 100
data['bp_interaction'] = data['systolic_bp'] * data['diastolic_bp'] / 100

# Create risk scores
data['metabolic_syndrome_score'] = (
    (data['waist_circumference'] > 102) | (data['gender'] == 'Female') & (data['waist_circumference'] > 88)
).astype(int) + (data['triglycerides'] > 150).astype(int) + (data['hdl_cholesterol'] < 40).astype(int) + \
    (data['systolic_bp'] > 130).astype(int) + (data['fasting_glucose'] > 100).astype(int)

print("Feature engineering completed!")
print(f"New features added: {list(data.columns[-6:])}")

## 5. Data Preprocessing

In [ ]:
# Prepare features and target
X = data.drop('prediabetes_risk', axis=1)
y = data['prediabetes_risk']

# Handle categorical variables
categorical_columns = X.select_dtypes(include=['object', 'category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Encode categorical variables
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Scale numerical features
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")

## 6. Model Development

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

# Train and evaluate models
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Train model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = (y_pred == y_test).mean()
    auc = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'auc': auc,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"{name} - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    print("-" * 50)

## 7. Model Evaluation

In [ ]:
# Compare model performance
model_comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[model]['accuracy'] for model in results.keys()],
    'AUC': [results[model]['auc'] for model in results.keys()]
})

print("Model Performance Comparison:")
print(model_comparison.sort_values('AUC', ascending=False))

# Plot ROC curves
plt.figure(figsize=(10, 8))
for name, result in results.items():
    fpr, tpr, _ = roc_curve(y_test, result['probabilities'])
    plt.plot(fpr, tpr, label=f'{name} (AUC = {result["auc"]:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Detailed evaluation of best model
best_model_name = max(results.keys(), key=lambda x: results[x]['auc'])
best_model = results[best_model_name]

print(f"Best Model: {best_model_name}")
print("=" * 50)

# Classification report
print("Classification Report:")
print(classification_report(y_test, best_model['predictions']))

# Confusion matrix
cm = confusion_matrix(y_test, best_model['predictions'])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Risk', 'Risk'], 
            yticklabels=['No Risk', 'Risk'])
plt.title(f'Confusion Matrix - {best_model_name}')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 8. Feature Importance Analysis

In [ ]:
# Feature importance for tree-based models
if hasattr(best_model['model'], 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model['model'].feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(feature_importance)), feature_importance['importance'])
    plt.yticks(range(len(feature_importance)), feature_importance['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Feature Importance - {best_model_name}')
    plt.tight_layout()
    plt.show()
    
    print("Top 10 Most Important Features:")
    print(feature_importance.head(10))

## 9. Model Optimization

In [ ]:
# Hyperparameter tuning for the best model
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10]
    }
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
else:
    print("Skipping hyperparameter tuning for this model type")
    param_grid = None

if param_grid:
    print(f"Performing hyperparameter tuning for {best_model_name}...")
    
    # Use cross-validation for tuning
    grid_search = GridSearchCV(
        models[best_model_name],
        param_grid,
        cv=5,
        scoring='roc_auc',
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")
    
    # Update best model with optimized parameters
    best_model['model'] = grid_search.best_estimator_

## 10. Risk Stratification Analysis

In [ ]:
# Create risk stratification
probabilities = best_model['probabilities']

# Define risk categories
risk_categories = pd.cut(probabilities, bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], 
                        labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

# Analyze risk distribution
risk_analysis = pd.DataFrame({
    'Risk_Category': risk_categories,
    'Actual_Risk': y_test,
    'Predicted_Probability': probabilities
})

risk_summary = risk_analysis.groupby('Risk_Category').agg({
    'Actual_Risk': ['count', 'mean'],
    'Predicted_Probability': 'mean'
}).round(3)

print("Risk Stratification Analysis:")
print(risk_summary)

# Visualize risk distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
risk_counts = risk_analysis['Risk_Category'].value_counts()
plt.pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%')
plt.title('Distribution of Predicted Risk Categories')

plt.subplot(1, 2, 2)
actual_risk_by_category = risk_analysis.groupby('Risk_Category')['Actual_Risk'].mean()
plt.bar(actual_risk_by_category.index, actual_risk_by_category.values)
plt.title('Actual Risk Rate by Predicted Category')
plt.ylabel('Actual Risk Rate')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 11. Clinical Decision Support

In [ ]:
# Create clinical decision support function
def get_clinical_recommendations(risk_probability, age, bmi, glucose, hba1c):
    """Generate clinical recommendations based on risk prediction"""
    
    recommendations = []
    
    if risk_probability < 0.2:
        recommendations.append("Low risk - Continue annual screening")
    elif risk_probability < 0.4:
        recommendations.append("Moderate risk - Consider lifestyle interventions")
        if bmi > 25:
            recommendations.append("Focus on weight management")
        if glucose > 100:
            recommendations.append("Monitor fasting glucose levels")
    elif risk_probability < 0.6:
        recommendations.append("High risk - Implement intensive lifestyle changes")
        recommendations.append("Consider metformin therapy")
        recommendations.append("Increase monitoring frequency")
    else:
        recommendations.append("Very high risk - Immediate intervention required")
        recommendations.append("Consider pharmacological therapy")
        recommendations.append("Refer to endocrinologist")
    
    return recommendations

# Example clinical recommendations
print("Clinical Decision Support Examples:")
print("=" * 50)

example_cases = [
    {'risk': 0.15, 'age': 45, 'bmi': 24, 'glucose': 95, 'hba1c': 5.5},
    {'risk': 0.35, 'age': 55, 'bmi': 28, 'glucose': 105, 'hba1c': 5.8},
    {'risk': 0.65, 'age': 60, 'bmi': 32, 'glucose': 115, 'hba1c': 6.2}
]

for i, case in enumerate(example_cases, 1):
    print(f"\nCase {i}:")
    print(f"Risk Probability: {case['risk']:.1%}")
    print(f"Age: {case['age']}, BMI: {case['bmi']}, Glucose: {case['glucose']}, HbA1c: {case['hba1c']}")
    recommendations = get_clinical_recommendations(case['risk'], case['age'], case['bmi'], case['glucose'], case['hba1c'])
    print("Recommendations:")
    for rec in recommendations:
        print(f"  - {rec}")

## 12. Model Deployment Considerations

In [ ]:
# Model performance metrics for deployment
deployment_metrics = {
    'Model': best_model_name,
    'Accuracy': best_model['accuracy'],
    'AUC': best_model['auc'],
    'Sensitivity': cm[1, 1] / (cm[1, 0] + cm[1, 1]),
    'Specificity': cm[0, 0] / (cm[0, 0] + cm[0, 1]),
    'PPV': cm[1, 1] / (cm[0, 1] + cm[1, 1]),
    'NPV': cm[0, 0] / (cm[0, 0] + cm[1, 0])
}

print("Deployment-Ready Model Metrics:")
print("=" * 50)
for metric, value in deployment_metrics.items():
    if metric != 'Model':
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

# Model interpretability score
interpretability_scores = {
    'Logistic Regression': 5,
    'Random Forest': 4,
    'Gradient Boosting': 3,
    'SVM': 2
}

print(f"\nModel Interpretability Score: {interpretability_scores.get(best_model_name, 'N/A')}/5")

# Deployment checklist
print("\nDeployment Checklist:")
print("✓ Model performance meets clinical standards")
print("✓ Feature importance analysis completed")
print("✓ Risk stratification implemented")
print("✓ Clinical decision support rules defined")
print("✓ Synthetic data disclaimer included")
print("⚠ Ensure HIPAA compliance for real data")
print("⚠ Validate with domain experts")
print("⚠ Implement monitoring and retraining protocols")

## 13. Summary and Conclusions

In [ ]:
print("Prediabetes Risk Prediction Analysis - Summary")
print("=" * 60)
print(f"\nBest Performing Model: {best_model_name}")
print(f"AUC Score: {best_model['auc']:.4f}")
print(f"Accuracy: {best_model['accuracy']:.4f}")

print("\nKey Findings:")
print("• Model successfully identifies patients at risk for prediabetes")
print("• Feature importance analysis reveals key risk factors")
print("• Risk stratification enables targeted interventions")
print("• Clinical decision support rules provide actionable recommendations")

print("\nClinical Impact:")
print("• Early identification of at-risk patients")
print("• Personalized intervention strategies")
print("• Improved resource allocation")
print("• Enhanced preventive care outcomes")

print("\nNext Steps:")
print("• Validate model with real clinical data")
print("• Implement in clinical workflow")
print("• Monitor model performance over time")
print("• Develop patient education materials")

print("\nNote: This analysis uses synthetic data for educational purposes.")
print("Real clinical implementation requires proper validation and regulatory compliance.")